###  모델 변환
#### 케라스 모델 변환
- 텐스플로의 tf.keras 모듈을 통해 케라스 모델을 바로 만들거나
- saveModel, HDF5포맷으로 저장된 모델을 케라스 모델로 불러와서 텐서플로 라이트 모델로 변환할 수 있다.
- 여기서는 다층 퍼셉트론 모델을 이용하여 변환하는 방법을 알아보겠다.

In [6]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

mlp_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2541 - accuracy: 0.9277
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1104 - accuracy: 0.9671
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0761 - accuracy: 0.9768
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0568 - accuracy: 0.9828
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0445 - accuracy: 0.9863


- mlp_model을 텐서플로 라이트 모델로 변환

In [8]:
# Keras 모델 변환
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\yourm\AppData\Local\Temp\tmpflwv_bnx\assets


INFO:tensorflow:Assets written to: C:\Users\yourm\AppData\Local\Temp\tmpflwv_bnx\assets


In [9]:
# 변환한 모델 tflite로 저장
with open('./keras_model_20220430_1.tflite', 'wb') as f:
  f.write(tflite_model)

#### savedModel 변환
- 텐서플로 모델을 저장하는 방법
- 1. 학습된 파라미터만 저장하는 방법
-  파라미터만 저장하려면 체크포인트를 사용 - 모델이 사용한 모든 파라미터 값을 저장하고 모델의 아키텍처는 저장하지 않는다.

- 2. 모델 전체를 저장하는 방법
-  모델 아키텍처, 가중치, 컴파일 관련 설정 값, 옵티마이저를 모두 저장해야하므로 
- 2-1 saveModle - 텐서플로 v2에서 권장
- 2-2 HDF5 포맷 - 텐서플로 v1 에서 주로 사용되던 방식

In [7]:
# SavedModel 저장
tf.saved_model.save(mlp_model, "./mlp_model/")
# 또는
mlp_model.save("./mlp_model/")

INFO:tensorflow:Assets written to: ./mlp_model/assets


INFO:tensorflow:Assets written to: ./mlp_model/assets


INFO:tensorflow:Assets written to: ./mlp_model/assets


INFO:tensorflow:Assets written to: ./mlp_model/assets


In [6]:
# HDF5 저장
mlp_model.save("./mlp_model.h5")

In [8]:
# SavedModel 불러오기
saved_model = tf.keras.models.load_model("./mlp_model/")

In [12]:
# # HDF5 불러오기
h5_model = tf.keras.models.load_model("./mlp_model.h5")

In [9]:
# SavedModel 변환
# 이방식 추천
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
tflite_model = converter.convert()

In [10]:
with open('./saved_model.tflite', 'wb') as f:
  f.write(tflite_model)

#### Concrete 함수 변환
- 텐서플로 v2에는 즉시 실행 모드가 추가되었으며 기본으로 활성화되어 있다.
- 모델을 고도화하기 위해 즉시 실행 모드보다 성능과 이식성이 뛰어난 그래프 모드를 사용하기도 한다.
- 즉시 실행 모드로 동작하는 파이썬 함수에 @tf.function 데코레이터를 붙이거나 
- 모델과 함수를 tf.function() 함수에 인자로 전달하면 자동으로 그래프 모드로 변환된다.


In [17]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs + 1
inc = Inc()


#### Inc 레이어의 call() 함수에 @tf.function 데코레이터를 추가하여 그래프 모드로 바꿀 수 있다.

In [18]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1
inc_g = Inc_Graph()

#### Inc클래서의 인스턴스를 그래프 모드로 변환

In [19]:
inc_g2 = tf.function(inc)


In [20]:
print(inc_g(tf.constant(3)))
print(inc_g(tf.constant([3, 2])))
print(inc_g(tf.constant([[3, 2], [1.0, 5.0]])))

print(inc_g2(tf.constant(3)))
print(inc_g2(tf.constant([3, 2])))
print(inc_g2(tf.constant([[3, 2], [1.0, 5.0]])))

tf.Tensor(4, shape=(), dtype=int32)


tf.Tensor([4 3], shape=(2,), dtype=int32)


tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)


#### concrete 함수 
- 텐스플로 그래프는 일반적인 파이썬 함수와 달리 정적인 데이터 타입과 형태가 필요하기 때문에 호출 시
- 전달받은 파라미터의 타입과 형태에 맞는 Concrete함수를 만든다.
- 타입과 형태를 묶어서 시그니처(signature)라고 한다.


In [21]:
print(inc_g(tf.constant(4)))
print(inc_g2(tf.constant(4)))

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


#### 시그니처별 Concrete함수는 get_concrete_function() 함수에 시그니처를 입력하여 얻을 수 있다.

In [27]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype=tf.float32))
#print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

#oncrete_fun = inc_g2.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


#### tf.function() 함수를 이용한 그래프 모드 적용

In [30]:
mlp_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

graph_model = tf.function(mlp_model)
concrete_func = graph_model.get_concrete_function(
  tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))


In [31]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('./concrete_func_model.tflite', 'wb') as f:
  f.write(tflite_model)

#### @tf.function 데코레이터를 이용한 그래프 모드 적용

In [32]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation='softmax')
    
    @tf.function
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)

mlp_model = MLP_Model()
concrete_func = mlp_model.call.get_concrete_function(
    tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))


In [33]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('./concrete_func_model.tflite', 'wb') as f:
  f.write(tflite_model)

### 텐스플로 허브의 TFLite모델
- 텐서플로 허브는 ffilte 형태로 변환이 완료된 모델도 제공
- 텐서플로 허브 메인 페이지에서 왼쪽 Model format 의 TFLite 를 선택 다운

### 기기배포
- 안드로이드 스튜디오에서 프로젝트에 assets폴더를 생성 - tflite파일을 복사 (배포완료..!!)
- 1) 모듈 이름을 마우스 오른쪽 버튼으로 클릭 [New] - [Folder] - [Assets Folder] 선택
- 2) assets 폴더를 생성할 수 있는 창이 나타나면 Finish 를 클릭
- 3) assets폴더 생성
- 4) assets폴더가 생성된 경로는 - Project root/모듈명/src/main/assets
- 5) 이 위치에 tflite 파일을 복사하거나 안드로이드 스튜디오에서 assets폴더에 바로  드래그앤드롭
- 6) assets 폴더 아래에 tflite 파일이 보이면 제대로 배포된것